<a href="https://colab.research.google.com/github/chricannon/Predicting_popular_platforms/blob/main/Audio_Features_Output_SpotifyAPI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import base64
import requests
import json



def get_token():
    auth_string = client_id + ":" + client_secret
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + auth_base64,
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {"grant_type": "client_credentials"}
    result = requests.post(url, headers=headers, data=data)
    json_result = json.loads(result.content)
    token = json_result["access_token"]
    return token

def get_auth_header(token):
    return {"Authorization": "Bearer " + token}

# Function to search for an artist
def search_for_artist(token, artist_name):
    url = "https://api.spotify.com/v1/search"
    headers = get_auth_header(token)
    query = f"?q={artist_name}&type=artist&limit=1"

    query_url = url + query
    result = requests.get(query_url, headers=headers)
    json_result = json.loads(result.content)["artists"]["items"]
    if len(json_result) == 0:
        print("No artist with this name exists...")
        return None

    return json_result[0]

# Function to get all albums, singles, and compilations by the artist
def get_albums_singles_compilations(token, artist_id):
    url = f"https://api.spotify.com/v1/artists/{artist_id}/albums?include_groups=album,single,compilation&limit=50"
    headers = get_auth_header(token)
    result = requests.get(url, headers=headers)
    json_result = json.loads(result.content)["items"]
    return json_result

# Function to get all tracks from an album
def get_tracks_from_album(token, album_id):
    url = f"https://api.spotify.com/v1/albums/{album_id}/tracks?limit=50"
    headers = get_auth_header(token)
    result = requests.get(url, headers=headers)
    json_result = json.loads(result.content)["items"]
    return json_result

# Function to get tracks where the artist is featured
def get_featured_tracks(token, artist_id):
    url = f"https://api.spotify.com/v1/artists/{artist_id}/albums?include_groups=appears_on&limit=50"
    headers = get_auth_header(token)
    result = requests.get(url, headers=headers)
    json_result = json.loads(result.content)["items"]
    return json_result

# Function to get top tracks by the artist
def get_top_tracks_by_artist(token, artist_id):
    url = f"https://api.spotify.com/v1/artists/{artist_id}/top-tracks?country=US"
    headers = get_auth_header(token)
    result = requests.get(url, headers=headers)
    json_result = json.loads(result.content)["tracks"]
    return json_result

# Function to filter tracks to ensure they include the artist as a primary artist
def filter_tracks_by_artist(tracks, artist_name):
    filtered_tracks = []
    for track in tracks:
        for artist in track["artists"]:
            if artist["name"].lower() == artist_name.lower():
                filtered_tracks.append(track)
                break
    return filtered_tracks

# Function to remove duplicate tracks
def remove_duplicates(tracks):
    unique_tracks = []
    seen_track_ids = set()
    for track in tracks:
        track_id = track['id']
        if track_id not in seen_track_ids:
            unique_tracks.append(track)
            seen_track_ids.add(track_id)
    return unique_tracks

# Function to get audio features for a list of track IDs, respecting the 100 IDs limit per request
def get_audio_features(token, track_ids):
    url = f"https://api.spotify.com/v1/audio-features"
    headers = get_auth_header(token)
    audio_features = []
    for i in range(0, len(track_ids), 100):
        chunk = track_ids[i:i + 100]
        params = {"ids": ",".join(chunk)}
        print(f"Requesting audio features for track IDs: {params}")  # Add logging
        result = requests.get(url, headers=headers, params=params)
        if result.status_code != 200:
            print(f"Failed to get audio features: {result.status_code}, {result.content}")
            continue
        json_result = json.loads(result.content)
        if 'audio_features' not in json_result:
            print(f"Key 'audio_features' not found in response: {json_result}")
            continue
        audio_features.extend(json_result['audio_features'])
    return audio_features

# Function to get tracks with their audio features
def get_tracks_with_features(token, tracks):
    track_ids = [track['id'] for track in tracks]  # Define track_ids here
    audio_features = get_audio_features(token, track_ids)  # Use track_ids as an argument
    return [(track, feature) for track, feature in zip(tracks, audio_features) if feature is not None]

token = get_token()
result = search_for_artist(token, "John Summit")
print(result["name"])
artist_id = result["id"]

# Get top tracks by the artist
top_tracks = get_top_tracks_by_artist(token, artist_id)

# Print top tracks
print("Top Tracks:")
for idx, song in enumerate(top_tracks):
    print(f"{idx + 1}. {song['name']}")

# Get all albums, singles, and compilations by the artist
albums_singles_compilations = get_albums_singles_compilations(token, artist_id)

# Get all tracks from each album, single, and compilation
all_tracks = []
for item in albums_singles_compilations:
    album_id = item["id"]
    tracks = get_tracks_from_album(token, album_id)
    filtered_tracks = filter_tracks_by_artist(tracks, "John Summit")
    all_tracks.extend(filtered_tracks)

# Get all tracks where the artist is featured
featured_albums = get_featured_tracks(token, artist_id)
for item in featured_albums:
    album_id = item["id"]
    tracks = get_tracks_from_album(token, album_id)
    filtered_tracks = filter_tracks_by_artist(tracks, "John Summit")
    all_tracks.extend(filtered_tracks)

# Remove duplicate tracks
unique_tracks = remove_duplicates(all_tracks)  # Apply remove_duplicates here

# Get audio features for unique tracks
tracks_with_features = get_tracks_with_features(token, unique_tracks)  # Get audio features

# Print all unique tracks with their audio features
print("\nTracks with Audio Features:")
for idx, (track, feature) in enumerate(tracks_with_features):
    print(f"{idx + 1}. {track['name']} - {feature}")


John Summit
Top Tracks:
1. Where You Are
2. Go Back (feat. Julia Church)
3. Shiver
4. Sweet Disposition - John Summit & Silver Panda Remix
5. Tears (with Paige Cavell)
6. La Danza
7. Resonate (feat. Julia Church)
8. Deep End - SIDEPIECE Remix
9. Better Than This
10. What A Life (feat. Stevie Appleton)
Requesting audio features for track IDs: {'ids': '2pObpUTLvvhg8PxSid23gG,2NKwz3gyK8da44pR2ZbiM1,6NPuaE61QbNqRb6vZxe8lZ,32VIrOsJmwvqRm4rWFBCsi,0TQWYkX6e29yzKUvUa2Ydg,5NC3MxGUpRR92X0fyX6946,4ZmWVoFNiBj0iNnnC7E4MA,4H0klYMkrrII3U6wBX6UuX,3pUz2qJe5nqZemi3hhIxMk,6PirrekW8pomBQH3A3XgGT,4CfcXnwLf2MUfnjCnqUMvj,6y5HLopYu7Uu0hYwVBj4T6,2pvQCbUnXHOpAcJOkgAYmZ,3wPPekju1lN3XgdtYCWmOS,26KJeqvcDJo6RtL0a6wyF2,68R0zVUeMJ2C852Ov6d2Mh,1fYuYXekH1FjjTSzbkfRHq,0y5AwOPNSipCieOBz7gvV8,6rGf5AsEaVskU0pO2ewcUL,1HmeANWuuCP6ldvGgKwTLi,52jCFwQBAf04Bi8xrE8DJu,5j6hNzexhw3Qm01mW7XlSq,6gs5VDiNpbBfXHsAj2c9SB,6hI3PnULFXwkOQSHLHNfOO,5pSUxTYDsOoGhzyCW4aXCL,3zFsJdVCu29em7VUB3Ujqp,6vc0R5eIkOBmqHw2CSDDBw,7pUeNT1OrNxhAQgeBTvihk,47k